In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
PATH="../input"
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
import os
print(os.listdir("../input"))
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
# this work is for practice, inspired by many excellent previous works 
# https://www.kaggle.com/gpreda/donorschoose-extensive-eda
# https://www.kaggle.com/ranliu/donor-project-matching-with-recommender-systems
# https://www.kaggle.com/gunnvant/building-content-recommender-tutorial
# https://www.kaggle.com/mahbubrob/donorschoose-full-eda-source-destination-map
# https://www.kaggle.com/shivamb/a-visual-and-insightful-journey-donorschoose

In [ ]:
donations = pd.read_csv(PATH+"/io/Donations.csv")
donors = pd.read_csv(PATH+"/io/Donors.csv", low_memory=False)
projects = pd.read_csv(PATH+"/io/Projects.csv", error_bad_lines=False, warn_bad_lines=False)
resources = pd.read_csv(PATH+"/io/Resources.csv", error_bad_lines=False, warn_bad_lines=False)
schools = pd.read_csv(PATH+"/io/Schools.csv", error_bad_lines=False)
teachers = pd.read_csv(PATH+"/io/Teachers.csv", error_bad_lines=False)

In [ ]:
resources.head(3)

In [ ]:
donations.head(1)

In [ ]:
## display files details
pd.DataFrame({'Dataset':['Donations','Donors','Resources','Schools','Teachers', 'Projects'],
             'Datapoints':[donations.shape[0], donors.shape[0], resources.shape[0],
                     schools.shape[0], teachers.shape[0], projects.shape[0]],
             'Features':[donations.shape[1], donors.shape[1], resources.shape[1],
                     schools.shape[1], teachers.shape[1], projects.shape[1]]})

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

## Check for missing data

In [ ]:
missing_data(projects)
# Deal with missing values

In [ ]:
missing_data(donations)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings("ignore")

## Teachers

In [ ]:
#check for duplicates for unique IDs
((teachers['Teacher ID'].value_counts().values)>1).any()
#((donors['Donor ID'].value_counts().values)>1).any()
#((donations['Donation ID'].value_counts().values)>1).any()

In [ ]:
# found duplicates in donations, checked details finding received dates have duplicates
#pd.Series(donations['Donation ID'])[pd.Series(donations['Donation ID']).duplicated()].values
#donations.loc[donations['Donation ID'] == '3aba50dc6813f02a1de167e8556963be']
donations=donations.drop_duplicates(subset='Donation ID', keep=False)

In [ ]:
teachers.head(1)

In [ ]:
##### check distributions for teacher prefix
tmp = teachers['Teacher Prefix'].value_counts()
df1 = pd.DataFrame({'Teacher Prefix': tmp.index,'Number of teachers': tmp.values})
fig, (ax1) = plt.subplots(ncols=1, figsize=(6,6))
s = sns.barplot(ax = ax1, x = 'Teacher Prefix', y="Number of teachers",data=df1)
plt.show();

In [ ]:
#df1
#female:347903 male:47480
#ratio
347903/(347903+47480)

In [ ]:
## distribution of months
teachers['Teacher First Project Posted Date'] = pd.to_datetime(
    teachers['Teacher First Project Posted Date'], errors='coerce')
teachers['weekdays'] = teachers['Teacher First Project Posted Date'].dt.dayofweek

teachers['month'] = teachers['Teacher First Project Posted Date'].dt.month

teachers['year'] = teachers['Teacher First Project Posted Date'].dt.year

weekdays = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',
            4:'Friday',5:'Saturday',6:'Sunday'}

months= {1 :"01",2 :"02",3 :"03",4 :"04",5 : "05",6 : "06",
          7 : "07",8 :"08", 9 :"09",10 :"10",11 :"11",12 :"12"}
#months= {1 :"Jan",2 :"Feb",3 :"Mar",4 :"Apr",5 : "May",6 : "Jun",
#          7 : "Jul",8 :"Aug", 9 :"Sep",10 :"Oct",11 :"Nov",12 :"Dec"}
teachers['weekdays']=teachers['weekdays'].map(weekdays)
teachers['month']=teachers['month'].map(months)

plt.figure(figsize=(10,6))
plt.bar(teachers['month'].value_counts().index, 
        teachers['month'].value_counts(),
        color=sns.color_palette("husl",12))
plt.xlabel('Months')
plt.ylabel('Counts')
plt.title('Popular Months for Creating Projects')
plt.tight_layout()

In [ ]:
## project creation trend
ts = teachers.groupby('year').agg({'Teacher ID' : 'count'}).reset_index()
plt.figure(figsize=(10,6))
plt.plot(ts['year'][:-1],ts['Teacher ID'][:-1], 
         marker = 'o', color=sns.color_palette('plasma')[2] )
plt.xlabel('Years')
plt.ylabel('Number of projects')
plt.title('Teacher Project Creation Trend')
plt.tight_layout()

In [ ]:
gender = {'Mrs.':'Female','Ms.':'Female','Mr.':'Male','Dr.':'Unknown','Mx.':'Unknown'}
teachers['gender'] = teachers['Teacher Prefix']
teachers['gender'] = teachers['gender'].map(gender)
plt.figure(figsize=(10,6))
genders = ['Female','Male','Unknown']
for i in range(len(genders)):
    ts = teachers[teachers['gender'] == genders[i]].groupby('year').agg({'Teacher ID' : 'count'}).reset_index()
    
    plt.plot(ts['year'][:-1],ts['Teacher ID'][:-1], marker = 'o',markersize = 8,
             color=sns.color_palette('husl')[i], label = genders[i])
    plt.xlabel('Year')
    plt.ylabel('Number of projects created')
    
plt.legend()
plt.title('Project Creation Trend, by Gender')
plt.tight_layout()

## Schools

In [ ]:
schools.head(1)

In [ ]:
projects.head(1)

In [ ]:
projects.iloc[0,6]

In [ ]:
tmp = schools['School Metro Type'].value_counts()
df1 = pd.DataFrame({'School Metro Type': tmp.index,'Counts': tmp.values})
fig, (ax1) = plt.subplots(ncols=1, figsize=(6,6))
s = sns.barplot(ax = ax1, x = 'School Metro Type', y="Counts",data=df1)
plt.show();

In [ ]:
# free lunch
schools_give_lunch = schools[['School Percentage Free Lunch', 'School State']]
schools_give_lunch.groupby('School State')['School Percentage Free Lunch'].describe().sort_values(by='mean', ascending=False).head(5)

In [ ]:
# free lunch
plt.figure(figsize=(10,6))
schools['School Percentage Free Lunch'].hist(bins = 50, color=sns.color_palette('husl')[4])
plt.xlabel('School Percentage Free Lunch')
plt.ylabel('Counts')
plt.title('School Percentage Free Lunch Histogram')
plt.tight_layout()

In [ ]:
##school&Projects states

projects_schools = projects.merge(schools, on='School ID', how='inner')

tmp = projects_schools['School State'].value_counts()
df1 = pd.DataFrame({'School State': tmp.index,'Number of projects': tmp.values})
fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'School State', y="Number of projects",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.title('Number of Projects by State')

plt.show();

## Resources

In [ ]:
tmp = resources.groupby('Resource Vendor Name')['Resource Quantity'].sum().sort_values(ascending = False).head(20)
df1 = pd.DataFrame({'Resource Vendor Name': tmp.index,'Number of resources': tmp.values})
fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'Resource Vendor Name', y="Number of resources",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show();

In [ ]:
tmp = resources['Resource Vendor Name'].value_counts().head(20)
df1 = pd.DataFrame({'Resource Vendor Name': tmp.index,'Number of orders': tmp.values})
fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'Resource Vendor Name', y="Number of orders",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show();

## Donors

In [ ]:
donors.head(20)

In [ ]:
#donors state
tmp = donors['Donor State'].value_counts()
df1 = pd.DataFrame({'Donor State': tmp.index,'Number of donors': tmp.values})

fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'Donor State', y="Number of donors",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.title('Number of donors by State')

plt.show();

In [ ]:
#donors state
tmp = donors['Donor State'].value_counts()
df1 = pd.DataFrame({'Donor State': tmp.index,'Number of donors': tmp.values})
pop = pd.read_csv(PATH+"/pop-2017/pop 2017.csv")
df1 = df1.merge(pop, how = "inner", left_on="Donor State", right_on = "State")
df1['Donors per 100,000']=(df1['Number of donors']/df1['2017'])*100000
fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'Donor State', y="Donors per 100,000",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.title('Donor rate (per 100,000) by State')

plt.show();

In [ ]:
df1.sort_values(by='Donors per 100,000', ascending=False).head(5)

In [ ]:
plt.figure(figsize=(10,14))
plt.barh(np.arange(0, 240, 16),
        donors['Donor City'].value_counts()[:15][::-1], height = 12, 
         color=sns.color_palette('husl'), align='center')
plt.yticks(np.arange(0, 256, 16), donors['Donor City'].value_counts()[:15][::-1].index)
plt.xlabel('Counts')
plt.ylabel('City')
plt.title('Donors City Distribution')
plt.tight_layout()

In [ ]:
# donors teacher
plt.figure(figsize=(10,6))
plt.bar(donors['Donor Is Teacher'].value_counts().index, 
        donors['Donor Is Teacher'].value_counts(),
        color=sns.color_palette('husl'))
plt.xlabel('Donor Is Teacher')
plt.ylabel('Counts')
plt.title('Donor-Teacher Distribution')
plt.tight_layout()

In [ ]:
# repeated donors
print('Total Donation Received')
print(donors.shape[0])
repeating_donors=donations['Donor ID'].value_counts().to_frame()
print('Total Number of Repeating Donors')
print(repeating_donors[repeating_donors['Donor ID']>1].shape[0])

In [ ]:
donors_schools = donations.merge(donors, left_on='Donor ID', right_on='Donor ID')

donors_schools = donors_schools.merge(projects[['Project ID', 'School ID']], 
                                            left_on='Project ID', right_on='Project ID')

donors_schools = donors_schools.merge(schools, left_on='School ID', right_on='School ID')

donors_schools.head(3)

In [ ]:
top_n = 10  # How many to look at
top_donor_states = donors_schools.groupby('Donor State')['Donation Amount'].sum().sort_values(ascending=False)
#top_donor_states.drop('other', inplace=True)  # Don't plot other, not interesting for now
top_donor_states = top_donor_states[:top_n]
top_donor_states = top_donor_states/1000000
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
sns.barplot(y=top_donor_states.index, x=top_donor_states, ax=ax, palette=sns.color_palette('husl', top_n))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)  
ax.set_xlabel('Donation Amount ($1,000,000)')
ax.set_title('Donation Amount by Donor State')
plt.show()

In [ ]:
top_donor_states/1000000

In [ ]:
# Limit data to include only those states that contain both donors and schools
from sklearn.preprocessing import StandardScaler, LabelEncoder

school_states = donors_schools['School State'].unique()
donor_states = donors_schools['Donor State'].unique()
states_to_keep_mask = [x in school_states for x in donor_states]
states = donor_states[states_to_keep_mask]
donors_schools = donors_schools[donors_schools['School State'].isin(states)]
donors_schools = donors_schools[donors_schools['Donor State'].isin(states)]
# Pivot, summing the donations
pivot = donors_schools.pivot_table(columns='School State',
                                      index='Donor State', 
                                      values='Donation Amount', 
                                      aggfunc='sum',
                                      fill_value=0)
# Separate the top n donors
top_n_pivot = pivot.loc[top_donor_states.index, :]
# Remove any states that none of them donate too
top_n_pivot = top_n_pivot.loc[:, top_n_pivot.sum() > 0]
# Unpivot
donation_paths = top_n_pivot.reset_index().melt(id_vars='Donor State')
donation_paths = donation_paths[donation_paths['value'] > 200000]  # Only significant amounts #can be changed

# Encode state names to integers for the Sankey
donor_encoder, school_encoder = LabelEncoder(), LabelEncoder()
donation_paths['Encoded Donor State'] = donor_encoder.fit_transform(donation_paths['Donor State'])
donation_paths['Encoded School State'] = school_encoder.fit_transform(donation_paths['School State']) + len(donation_paths['Encoded Donor State'].unique())
# Create a state to color dictionary
all_states = np.unique(np.array(donation_paths['School State'].unique().tolist() + donation_paths['Donor State'].unique().tolist()))
#plotly_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
plotly_colors = ['#F27420','#4994CE','#FABC13','#7FC241','#9467bd', '#8c564b', '#e377c2','#D3D3D3','#0e85f5', '#2ca02c']

In [ ]:
states_finished = False
state_colors = []
i = 0
while not states_finished:
    
    state_colors.append(plotly_colors[i]) 
    
    if len(state_colors) >= len(all_states):
        states_finished = True
        
    i += 1
    if i >= len(plotly_colors):
        i = 0
        
color_dict = dict(zip(all_states, state_colors))

snky_labels = donor_encoder.classes_.tolist()  + school_encoder.classes_.tolist()
colors = []
for state in snky_labels:
    colors.append(color_dict[state])

data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(
        color = "blue",
        width = 0.5
      ),
      label = snky_labels,
      color = colors,
    ),
    link = dict(
      source = donation_paths['Encoded Donor State'],
      target = donation_paths['Encoded School State'],
      value = donation_paths['value'],
  ))

layout =  dict(
    title = "Donation flow (amount more than $200,000)",
    autosize=False,
    width=800,
    height=750,

    font = dict(
      size = 10
    )
)
py.init_notebook_mode(connected=True)
fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

school_states = donors_schools['School State'].unique()
donor_states = donors_schools['Donor State'].unique()
states_to_keep_mask = [x in school_states for x in donor_states]
states = donor_states[states_to_keep_mask]
donors_schools = donors_schools[donors_schools['School State'].isin(states)]
donors_schools = donors_schools[donors_schools['Donor State'].isin(states)]
# Pivot, summing the donations
pivot = donors_schools.pivot_table(columns='School State',
                                      index='Donor State', 
                                      values='Donation Amount', 
                                      aggfunc='sum',
                                      fill_value=0)
# Separate the top n donors
top_n_pivot = pivot.loc[top_donor_states.index, :]
# Remove any states that none of them donate too
top_n_pivot = top_n_pivot.loc[:, top_n_pivot.sum() > 0]
# Unpivot
donation_paths = top_n_pivot.reset_index().melt(id_vars='Donor State')
donation_paths = donation_paths[donation_paths['value'] > 100000]  # Only significant amounts #can be changed

# Encode state names to integers for the Sankey
donor_encoder, school_encoder = LabelEncoder(), LabelEncoder()
donation_paths['Encoded Donor State'] = donor_encoder.fit_transform(donation_paths['Donor State'])
donation_paths['Encoded School State'] = school_encoder.fit_transform(donation_paths['School State']) + len(donation_paths['Encoded Donor State'].unique())
# Create a state to color dictionary
all_states = np.unique(np.array(donation_paths['School State'].unique().tolist() + donation_paths['Donor State'].unique().tolist()))
#plotly_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
plotly_colors = ['#F27420','#4994CE','#FABC13','#7FC241','#9467bd', '#8c564b', '#e377c2','#D3D3D3','#0e85f5', '#2ca02c']
states_finished = False
state_colors = []
i = 0
while not states_finished:
    
    state_colors.append(plotly_colors[i]) 
    
    if len(state_colors) >= len(all_states):
        states_finished = True
        
    i += 1
    if i >= len(plotly_colors):
        i = 0
        
color_dict = dict(zip(all_states, state_colors))

snky_labels = donor_encoder.classes_.tolist()  + school_encoder.classes_.tolist()
colors = []
for state in snky_labels:
    colors.append(color_dict[state])

data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(
        color = "blue",
        width = 0.5
      ),
      label = snky_labels,
      color = colors,
    ),
    link = dict(
      source = donation_paths['Encoded Donor State'],
      target = donation_paths['Encoded School State'],
      value = donation_paths['value'],
  ))

layout =  dict(
    title = "Donation flow (amount more than $100,000)",
    autosize=False,
    width=800,
    height=750,

    font = dict(
      size = 10
    )
)
py.init_notebook_mode(connected=True)
fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)

## Donations

In [ ]:
donations.head(1)

In [ ]:
print('Maximum Donation Amount: $', donations['Donation Amount'].max())
print('Minimum Donation Amount: $', donations['Donation Amount'].min())
print('Average Donation Amount: $', donations['Donation Amount'].mean())
print('Median Donation Amount: $', donations['Donation Amount'].median())

In [ ]:
donations['Donation Received Date'] = pd.to_datetime(
    donations['Donation Received Date'], errors='coerce')

donations['year'] = donations['Donation Received Date'].dt.year
donations['day-formated'] = donations['Donation Received Date'].dt.strftime('%m/%d/%Y')

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(donations['Donation Included Optional Donation'].value_counts().index, 
        donations['Donation Included Optional Donation'].value_counts(),
        color=['#FABC13','#7FC241'])
plt.xlabel('Donation Included Optional Donation')
plt.ylabel('Counts')
plt.title('Donations Included Optional Donation Distribution')
plt.tight_layout()

In [ ]:
donations['Donation Included Optional Donation'].value_counts()/donations.shape[0]

In [ ]:
# Extract Year, Month, Date, Time
donations["Donation Received Year"]   = donations["Donation Received Date"].dt.year
donations["Donation Received Month"]  = donations["Donation Received Date"].dt.month
donations["Donation Received Day"]    = donations["Donation Received Date"].dt.day
donations["Donation Received Hour"]   = donations["Donation Received Date"].dt.hour
temp = donations["Donation Received Year"].value_counts().head()

In [ ]:
temp =temp.sort_values(ascending=True)
plt.figure(figsize=(10,6))
temp.plot.bar(color=sns.color_palette('husl'))
plt.xlabel('Year')
plt.ylabel('Number of Donations') 
plt.title('Donations Received by Year')
plt.tight_layout()

In [ ]:
## donation + donors
donations_donors = donations.merge(donors, on='Donor ID', how='inner')

In [ ]:
tmp = donations_donors.groupby('Donor State')['Donation Amount'].sum()
df1 = pd.DataFrame({'State': tmp.index,'Total sum of donations': tmp.values})
df1['Total sum of donations']= df1['Total sum of donations']/1000000
df1.sort_values(by='Total sum of donations',ascending=False, inplace=True)
fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'State', y="Total sum of donations",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
ax1.set_ylabel("Donation $1,000,000")
ax1.set_title('Sum of Donations by State')
plt.show();

In [ ]:
# donation sum by state
tmp = donations_donors.groupby('Donor State')['Donation Amount'].sum()
df1 = pd.DataFrame({'State': tmp.index,'Total sum of donations': tmp.values})
df1 = df1.merge(pop, how = "inner", on = "State")
df1['Donation Amount per 100,000']=(df1['Total sum of donations']/df1['2017'])*100000

#df1.sort_values(by='Donation Amount per 100,000',ascending=False, inplace=True)  #descending order
fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
s = sns.barplot(ax = ax1, x = 'State', y="Donation Amount per 100,000",data=df1)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
ax1.set_title('Donation Amount per 100,000 by State')
plt.show();

## Projects

In [ ]:
projects.head(1)

In [ ]:
projects['Project Posted Date'] = pd.to_datetime(
    projects['Project Posted Date'], errors='coerce')

projects['Project Expiration Date'] = pd.to_datetime(
    projects['Project Expiration Date'], errors='coerce')

projects['Project Fully Funded Date'] = pd.to_datetime(
    projects['Project Fully Funded Date'], errors='coerce')

In [ ]:
projects['year-posted'] = projects['Project Posted Date'].dt.year
projects['day-posted-formated'] = projects['Project Posted Date'].dt.strftime('%m/%d/%Y')

projects['year-expiry'] = projects['Project Expiration Date'].dt.year
projects['day-expiry-formated'] = projects['Project Expiration Date'].dt.strftime('%m/%d/%Y')

projects['year-funded'] = projects['Project Fully Funded Date'].dt.year
projects['day-funded-formated'] = projects['Project Fully Funded Date'].dt.strftime('%m/%d/%Y')

projects['delta-days-before-expiry'] = (projects['Project Expiration Date'] - projects['Project Posted Date']).dt.days
projects['delta-days-before-funded'] = (projects['Project Fully Funded Date'] - projects['Project Posted Date']).dt.days

In [ ]:
def plot_wordcloud(feature,additionalStopWords=""):
    stopwords = set(STOPWORDS)
    stopwords.update(additionalStopWords)
    text = " ".join(projects_schools[feature][~pd.isnull(projects_schools[feature])].sample(50000))
    wordcloud = WordCloud(background_color='#fefefe',stopwords=stopwords,
                          max_words=300,max_font_size=50,width=600, height=400, random_state=666,
                         ).generate(text)
    fig = plt.figure(figsize = (12,12))
    plt.imshow(wordcloud)
    plt.title("Wordcloud with %s content" % feature, fontsize=16)
    plt.axis('off')
    plt.show()

In [ ]:
projects.head(2)

In [ ]:
# 5,6,7,8  title,essay,short description,need statement
#essay [0,6]
projects.iloc[867,5]  

In [ ]:
plot_wordcloud('Project Title', additionalStopWords = ["teacher", "donor", "students", "school", "will"])

In [ ]:
plot_wordcloud('Project Short Description', additionalStopWords = ["teacher", "donor", "students", "school", "will","<!--DONOTREMOVEESSAYDIVIDER-->"])

In [ ]:
plot_wordcloud('Project Need Statement', additionalStopWords = ["teacher", "donor", "students", "school", "will","<!--DONOTREMOVEESSAYDIVIDER-->","need"])

In [ ]:
def plot_category(feature):
    tmp = projects[feature].value_counts().sort_values(ascending = False).head(20)
    df1 = pd.DataFrame({feature: tmp.index,'Number of projects': tmp.values})
    fig, (ax1) = plt.subplots(ncols=1, figsize=(12,6))
    s = sns.barplot(ax = ax1, x = feature, y="Number of projects",data=df1)
    s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.show();

In [ ]:
plot_category('Project Subject Category Tree')

In [ ]:
plot_category('Project Subject Subcategory Tree')

In [ ]:
plot_category('Project Grade Level Category')

In [ ]:
df = projects.head(10000)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [ ]:
from  sklearn.feature_extraction import text
Text=df['Project Short Description'].tolist()
tfidf=text.TfidfVectorizer(input=Text,ngram_range=(1, 2),min_df=0,stop_words="english")
matrix=tfidf.fit_transform(Text)
matrix.shape

In [ ]:
### Get Similarity Scores using cosine similarity### Get 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(matrix, matrix)


In [ ]:
df = df.reset_index()
titles = df['Project Title']
indices = pd.Series(df.index, index=df['Project Title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
df.head(20)

In [ ]:
get_recommendations('Stand Up to Bullying: Together We Can!').head(10)

In [ ]:
get_recommendations("Our Old Computer Can't Keep Up With Our Learning").head(10)

# Clustering

In [ ]:
## build clusters for donors
donors_schools.head(3)

In [ ]:
project_fordonor= projects[['Project ID','Project Resource Category', 'Project Grade Level Category','Project Subject Category Tree','Project Cost']]
df_cluster = donors_schools.merge(project_fordonor, on = 'Project ID', how = 'inner')

In [ ]:
df_cluster = df_cluster[['Donation ID','Donation Included Optional Donation','Donation Amount','Donor State','School Metro Type','School State','Project Resource Category', 'Project Grade Level Category','Project Subject Category Tree','Project Cost']]
df_cluster['Project Subject Category Tree'] = df_cluster['Project Subject Category Tree'].fillna(" ")
## aggregate the donors and their past donations in order to create their donor - profiles
# 'Project Title' : lambda x: ",".join(x),

In [ ]:
df_cluster.head(1)

In [ ]:
df_cluster2 = pd.get_dummies(df_cluster, drop_first=True, columns=['Donation Included Optional Donation','Donor State','School Metro Type','School State','Project Resource Category', 'Project Grade Level Category','Project Subject Category Tree'])


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_score
from sklearn.metrics import completeness_score
from sklearn import metrics
sse=[]
df_cluster2 = df_cluster2.iloc[:,1:].head(10000)
for k in range(2,30):
    kmeans = KMeans(n_clusters=k,max_iter=100, random_state= 666).fit(df_cluster2)
    sse.append(kmeans.inertia_)

In [ ]:
fig = plt.figure()
x = range(2, 30)
fig, ax = plt.subplots(1, 1, figsize=[10, 5])
lns1 = ax.plot(x, sse[0:], '-go', label = "sse",markersize=5)
ax.set_xlabel('Number of Clusters')
ax.set_ylabel('sse', color='g')
lns = lns1
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc="best")
plt.title("Choosing K ")
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=40, n_iter=300, random_state= 666)
tsne_results = tsne.fit_transform(df_cluster2)
x1 = tsne_results[:,0]
x2 = tsne_results[:,1]

In [ ]:
## plot with several k values
# k =3
import matplotlib

plt.subplot(221)
kmeans = KMeans(n_clusters=3,max_iter=1000, random_state= 666)
cluster_labels = kmeans.fit_predict(df_cluster2)
colors = ['red','green','blue']
plt.scatter(x1, x2, c=cluster_labels, cmap=matplotlib.colors.ListedColormap(colors), alpha=0.7, marker='.',edgecolor='k')

plt.title("K = 3 ")
# k =4
plt.subplot(222)
kmeans = KMeans(n_clusters=4,max_iter=1000, random_state= 666)
cluster_labels = kmeans.fit_predict(df_cluster2)
colors = ['red','green','blue','yellow']
plt.scatter(x1, x2, c=cluster_labels, cmap=matplotlib.colors.ListedColormap(colors), alpha=0.7, marker='.',edgecolor='k')

plt.title("K = 4")
# k =5
plt.subplot(223)
kmeans = KMeans(n_clusters=5,max_iter=1000, random_state= 666)
cluster_labels = kmeans.fit_predict(df_cluster2)
colors = ['red','green','blue','yellow','purple']
plt.scatter(x1, x2, c=cluster_labels, cmap=matplotlib.colors.ListedColormap(colors), alpha=0.7, marker='.',edgecolor='k')

plt.title("K = 5 ")
# k =6
plt.subplot(224)
kmeans = KMeans(n_clusters=6,max_iter=1000, random_state= 666)
cluster_labels = kmeans.fit_predict(df_cluster2)
colors = ['red','green','blue','yellow','purple','gray']
plt.scatter(x1, x2, c=cluster_labels, cmap=matplotlib.colors.ListedColormap(colors), alpha=0.7, marker='.',edgecolor='k')

plt.title("K = 6")
plt.subplots_adjust(top=0.92, bottom=0.02, left=0.10, right=0.95, hspace=0.8,
                    wspace=0.35)
plt.show()

## donor history recommendation

In [ ]:
import math
test_mode = True
import scipy
import random

In [ ]:
f=len(projects)
projects['project_id'] = np.nan
g = list(range(10,f+10))
g = pd.Series(g)
projects['project_id'] = g.values
# Merge datasets
donations = donations.merge(donors, on="Donor ID", how="left")
df = donations.merge(projects,on="Project ID", how="left")

# only load a few lines in test mode
if test_mode:
    df = df.head(10000)

donations_df = df
# Deal with missing values
donations["Donation Amount"] = donations["Donation Amount"].fillna(0)

# Define event strength as the donated amount to a certain project
donations_df['eventStrength'] = donations_df['Donation Amount']

def smooth_donor_preference(x):
    return math.log(1+x, 2)
    
donations_full_df = donations_df \
                    .groupby(['Donor ID', 'Project ID'])['eventStrength'].sum() \
                    .apply(smooth_donor_preference).reset_index()
        
# Update projects dataset
project_cols = projects.columns
projects = df[project_cols].drop_duplicates()


In [ ]:
donations_train_df, donations_test_df = train_test_split(donations_full_df,
                                   test_size=0.30,
                                   random_state=666)

print('# donations on Train set: %d' % len(donations_train_df))
print('# donations on Test set: %d' % len(donations_test_df))

#Indexing by Donor Id to speed up the searches during evaluation
donations_full_indexed_df = donations_full_df.set_index('Donor ID')
donations_train_indexed_df = donations_train_df.set_index('Donor ID')
donations_test_indexed_df = donations_test_df.set_index('Donor ID')

In [ ]:
# Preprocessing of text data
textfeats = ["Project Title","Project Essay"]
for cols in textfeats:
    projects[cols] = projects[cols].astype(str) 
    projects[cols] = projects[cols].astype(str).fillna('') # FILL NA
    projects[cols] = projects[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
 
text = projects["Project Title"] + ' ' + projects["Project Essay"]
vectorizer = TfidfVectorizer(strip_accents='unicode',
                             analyzer='word',
                             lowercase=True, # Convert all uppercase to lowercase
                             stop_words='english', # Remove commonly found english words ('it', 'a', 'the') which do not typically contain much signal
                             max_df = 0.9, # Only consider words that appear in fewer than max_df percent of all documents
                             # max_features=5000 # Maximum features to be extracted                    
                            )                        
project_ids = projects['Project ID'].tolist()
tfidf_matrix = vectorizer.fit_transform(text)
tfidf_feature_names = vectorizer.get_feature_names()


In [ ]:
def get_project_profile(project_id):
    idx = project_ids.index(project_id)
    project_profile = tfidf_matrix[idx:idx+1]
    return project_profile

def get_project_profiles(ids):
    project_profiles_list = [get_project_profile(x) for x in np.ravel([ids])]
    project_profiles = scipy.sparse.vstack(project_profiles_list)
    return project_profiles

def build_donors_profile(donor_id, donations_indexed_df):
    donations_donor_df = donations_indexed_df.loc[donor_id]
    donor_project_profiles = get_project_profiles(donations_donor_df['Project ID'])
    donor_project_strengths = np.array(donations_donor_df['eventStrength']).reshape(-1,1)
    #Weighted average of project profiles by the donations strength
    donor_project_strengths_weighted_avg = np.sum(donor_project_profiles.multiply(donor_project_strengths), axis=0) / (np.sum(donor_project_strengths)+1)
    donor_profile_norm = sklearn.preprocessing.normalize(donor_project_strengths_weighted_avg)
    return donor_profile_norm

from tqdm import tqdm

def build_donors_profiles(): 
    donations_indexed_df = donations_full_df[donations_full_df['Project ID'].isin(projects['Project ID'])].set_index('Donor ID')
    donor_profiles = {}
    for donor_id in tqdm(donations_indexed_df.index.unique()):
        donor_profiles[donor_id] = build_donors_profile(donor_id, donations_indexed_df)
    return donor_profiles

donor_profiles = build_donors_profiles()


In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, projects_df=None):
        self.project_ids = project_ids
        self.projects_df = projects_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_projects_to_donor_profile(self, donor_id, topn=1000):
        #Computes the cosine similarity between the donor profile and all project profiles
        cosine_similarities = cosine_similarity(donor_profiles[donor_id], tfidf_matrix)
        #Gets the top similar projects
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar projects by similarity
        similar_projects = sorted([(project_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_projects
        
    def recommend_projects(self, donor_id, projects_to_ignore=[], topn=10, verbose=False):
        similar_projects = self._get_similar_projects_to_donor_profile(donor_id)
        #Ignores projects the donor has already donated
        similar_projects_filtered = list(filter(lambda x: x[0] not in projects_to_ignore, similar_projects))
        
        recommendations_df = pd.DataFrame(similar_projects_filtered, columns=['Project ID', 'recStrength']).head(topn)

        recommendations_df = recommendations_df.merge(self.projects_df, how = 'left', 
                                                    left_on = 'Project ID', 
                                                    right_on = 'Project ID')[['recStrength', 'Project ID', 'Project Title', 'Project Essay']]


        return recommendations_df

In [ ]:
#donations.iloc[568,:]  #random project and donor
print(df.shape[0])
repeating_donors=df['Donor ID'].value_counts().to_frame()
print('Total Number of Repeating Donors')
print(repeating_donors[repeating_donors['Donor ID']>1].shape[0])

In [ ]:
repeating_donors[repeating_donors['Donor ID']==3].head(5)

In [ ]:
df.head(1)

In [ ]:
df[df['Donor ID']=="fb961cd43218b42e06852a1e3bf82286"]
## 2 0072118edd7e0c5e84d23be2424bebd9  Our Learning Is On Fire!
## 1 0002a45d0b45a78e9c920beba40bc7fb  Fight the Summer Brain Drain With Books

In [ ]:
# repeat donors
mydonor = "fb961cd43218b42e06852a1e3bf82286"
model = ContentBasedRecommender(projects)
model.recommend_projects(mydonor)

In [ ]:
mydonor1 = "6d5b22d39e68c656071a842732c63a0c"
model.recommend_projects(mydonor1)

In [ ]:
mydonor2 = "8e97afb4cb30d655465da4affdcea329"
model.recommend_projects(mydonor2)